In [3]:
import numpy as np
import random
import time

def lire_benchmark(fichier):
    with open(fichier, "r") as f:
        lignes = f.readlines()

    # Lire les deux premiers nombres (n et m)
    n, m = map(int, lignes[0].strip().split())  

    # Lire les poids (2ème ligne du fichier)
    poids = np.array(list(map(int, lignes[1].strip().split())))

    # Lire les sous-ensembles restants
    subsets = []
    for ligne in lignes[2:]:
        subsets.append(list(map(int, ligne.strip().split())))

    return n, m, poids, subsets

# Fonction fitness : Maximiser la couverture
def fitness(solution, subsets, poids):
    couverture = set()
    for i, sel in enumerate(solution):
        if sel == 1:
            couverture.update(subsets[i])
    return len(couverture), sum(poids[i] for i in range(len(solution)) if solution[i] == 1)

# Algorithme PSO
def pso(n, m, subsets, poids, nb_iterations, W, c1, c2, Vmax, nb_particules):
    # Initialisation
    particules = [np.random.randint(0, 2, m) for _ in range(nb_particules)]
    vitesses = [np.random.uniform(-Vmax, Vmax, m) for _ in range(nb_particules)]
    
    # Meilleures solutions
    p_best = particules.copy()
    p_best_scores = [fitness(p, subsets, poids)[0] for p in particules]
    
    g_best = max(p_best, key=lambda p: fitness(p, subsets, poids)[0])
    g_best_score = fitness(g_best, subsets, poids)[0]
    
    # PSO main loop
    for _ in range(nb_iterations):
        for i in range(nb_particules):
            r1, r2 = random.random(), random.random()
            vitesses[i] = W * np.array(vitesses[i]) \
                          + c1 * r1 * (np.array(p_best[i]) - np.array(particules[i])) \
                          + c2 * r2 * (np.array(g_best) - np.array(particules[i]))
            vitesses[i] = np.clip(vitesses[i], -Vmax, Vmax)

            particules[i] = np.where(np.random.rand(m) < 1 / (1 + np.exp(-vitesses[i])), 1, 0)

            # Mise à jour des meilleures solutions
            score, poids_total = fitness(particules[i], subsets, poids)
            if score > p_best_scores[i]:
                p_best[i] = particules[i]
                p_best_scores[i] = score
                if score > g_best_score:
                    g_best = particules[i]
                    g_best_score = score

    return g_best_score

# Liste des hyperparamètres à tester
grid_search_params = {
    "nb_iterations": [50, 100, 150],
    "W": [0.5, 0.7, 0.9],
    "c1": [1.5, 2.0, 2.5],
    "c2": [1.5, 2.0, 2.5],
    "Vmax": [2, 4, 6],
    "nb_particules": [10, 20, 30]
}

# Lire le benchmark (fichier à spécifier)
fichier_benchmark = "Benchmark/scp41.txt"
n, m, poids, subsets = lire_benchmark(fichier_benchmark)

# Tester toutes les combinaisons et stocker les meilleurs paramètres
meilleur_score = 0
meilleurs_parametres = {}

for nb_iter in grid_search_params["nb_iterations"]:
    for W in grid_search_params["W"]:
        for c1 in grid_search_params["c1"]:
            for c2 in grid_search_params["c2"]:
                for Vmax in grid_search_params["Vmax"]:
                    for nb_part in grid_search_params["nb_particules"]:
                        start_time = time.time()
                        score = pso(n, m, subsets, poids, nb_iter, W, c1, c2, Vmax, nb_part)
                        exec_time = time.time() - start_time

                        # Affichage des paramètres testés
                        print(f"Test avec nb_iter={nb_iter}, W={W}, c1={c1}, c2={c2}, Vmax={Vmax}, nb_part={nb_part} → Score: {score} | Temps: {exec_time:.2f}s")

                        # Mise à jour des meilleurs paramètres
                        if score > meilleur_score:
                            meilleur_score = score
                            meilleurs_parametres = {
                                "nb_iterations": nb_iter,
                                "W": W,
                                "c1": c1,
                                "c2": c2,
                                "Vmax": Vmax,
                                "nb_particules": nb_part
                            }

# Affichage des meilleurs paramètres trouvés
print("\n🏆 Meilleurs paramètres trouvés :")
for k, v in meilleurs_parametres.items():
    print(f"{k}: {v}")
print(f"Meilleur score obtenu: {meilleur_score}")


IndexError: index 12 is out of bounds for axis 0 with size 12